In [ ]:
import numpy as np
import pandas as pd

from cyclops.processors.aggregate import Aggregator
from cyclops.processors.cleaning import (
    normalize_categories,
    normalize_names,
    normalize_values,
)
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    CARE_UNIT,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_CATEGORY,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    HOSPITAL_ID,
    SEX,
    SUBJECT_ID,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    BY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STANDARD,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.feature import TabularFeatures, TemporalFeatures
from cyclops.processors.feature.normalization import GroupbyNormalizer
from cyclops.processors.feature.split import split_datasets
from cyclops.query import mimic
from cyclops.query import process as qp
from cyclops.utils.common import print_dict
from cyclops.utils.file import load_dataframe, save_dataframe

In [ ]:
YEARS = [2015, 2016, 2018, 2019, 2020]
MIN_YEAR = min(YEARS)
TARGET_NAMES = ["death"]

# Querying

In [ ]:
encounters_interface = mimic.patient_encounters(
    years=YEARS, died=True, died_binarize_col="died"
)
encounters_query = encounters_interface.query
encounters_query = qp.Drop(
    ["insurance", "language", "marital_status", "hospital_expire_flag"]
)(encounters_query)
encounters_interface = mimic.get_interface(encounters_query)
encounters = encounters_interface.run()
encounters

In [ ]:
target_events = encounters[encounters["died"] == True]
target_events = target_events[[ENCOUNTER_ID, "deathtime"]]
target_events = target_events.rename({"deathtime": EVENT_TIMESTAMP}, axis=1)
target_events[EVENT_NAME] = TARGET_NAME
target_events[EVENT_VALUE] = 1
target_events.head(5)

In [ ]:
save_dataframe(encounters, "encounters.parquet")

In [ ]:
events_interface = mimic.events()
events_query = events_interface.query
events_query = qp.Drop(["warning", "itemid", "storetime"])(events_query)
events_interface = mimic.get_interface(events_query)
events = events_interface.run(limit=1000000)
events = pd.merge(
    encounters[[ENCOUNTER_ID, "anchor_year_difference"]], events, on=ENCOUNTER_ID
)


def add_offset(row):
    row[EVENT_TIMESTAMP] += pd.DateOffset(years=row["anchor_year_difference"])
    return row


events = events.apply(add_offset, axis=1)
events = events.drop("anchor_year_difference", axis=1)

# Include target as an event
events = pd.concat([events, target_events])

events

In [ ]:
save_dataframe(events, "events.parquet")

------------------------------------------------------------------------------------------------

# Processing

## Tabular

In [ ]:
encounters = load_dataframe("encounters.parquet")
encounters.head(5)

In [ ]:
features = [
    AGE,
    SEX,
    "admission_type",
    "admission_location",
    "discharge_location",
    "ethnicity",
]

tab_features = TabularFeatures(
    data=encounters,
    features=features,
    by=ENCOUNTER_ID,
)

In [ ]:
tab_features.get_data()

In [ ]:
tab_features.types

In [ ]:
tab_features.meta["admission_location"].get_mapping()

In [ ]:
ordinal_features = tab_features.features_by_type(ORDINAL)
ordinal_features

In [ ]:
tab_vectorized = tab_features.vectorize(to_binary_indicators=ordinal_features)
type(tab_vectorized)

In [ ]:
tab_vectorized.indexes[tab_vectorized.get_axis(FEATURES)]

In [ ]:
tab_vectorized.axis_names

In [ ]:
tab_vectorized.shape

In [ ]:
features, targets = tab_vectorized.split_out(FEATURES, [SEX, AGE])

In [ ]:
features.get_index(FEATURES)

In [ ]:
targets.get_index(FEATURES)

In [ ]:
tab_vectorized.split_by_fraction(BY, [0.8, 0.1])

## Temporal

In [ ]:
events = load_dataframe("events.parquet")

# Process
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_CATEGORY] = normalize_categories(events[EVENT_CATEGORY])
# events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

# Same event names are the same, but mean different things for different categories
# E.g., 'flow' for categories heartware and ecmo
events[EVENT_NAME] = events[EVENT_NAME] + " - " + events[EVENT_CATEGORY]
events.head(5)

In [ ]:
# Keep only the most popular events, including the target
top_events = events[EVENT_NAME].value_counts()[:150].index
top_events = np.append(top_events, TARGET_NAME).unique()
print(top_events)
events = events[events[EVENT_NAME].isin(top_events)]
events

In [ ]:
events.head(5)

In [ ]:
events[events[EVENT_NAME].str.contains("death")].head(5)

In [ ]:
aggregator = Aggregator(
    aggfuncs={EVENT_VALUE: MEAN},
    timestamp_col=EVENT_TIMESTAMP,
    time_by=ENCOUNTER_ID,
    agg_by=[ENCOUNTER_ID, EVENT_NAME],
    timestep_size=8,
    window_duration=24,  # Optional
    # agg_meta_for=EVENT_VALUE,  # Optional
)

In [ ]:
aggregated = tmp_features.aggregate()
aggregated.head(5)

In [ ]:
aggregated.head(100)

In [ ]:
save_dataframe(aggregated, "aggregated.parquet")

In [ ]:
temp_vectorized = aggregator.vectorize(aggregated)
temp_vectorized.data.shape

In [ ]:
temp_vectorized.axis_names

## Normalization

In [ ]:
tab_normalizer = GroupbyNormalizer({EVENT_VALUE: STANDARD}, by=EVENT_NAME)
tab_feature.add_normali

In [ ]:
feat_axis = vectorized.axis_names.index(FEATURES)
temp_normalizer = VectorizedNormalizer(
    feat_axis,
    normalization_method=STANDARD,
)

In [ ]:
# tmp_features.normalize(FEATURES)
# tmp_features.get_data()

In [ ]:
temp_vectorized = np.squeeze(temp_vectorized)
temp_vectorized.shape

In [ ]:
train_data, val_data, test_data = split_data(temp_vectorized, [0.7, 0.2])

In [ ]:
train_data.shape

In [ ]:
val_data.shape

In [ ]:
test_data.shape